This note book measures the grpc operation efficiency.

In [76]:
import sys
sys.path.append('../bfrt_grpc')
from bfrt_grpc_client import Bfrt_GRPC_Client
from bfrt_grpc_client import ip_to_int, int_to_ip
from bfrt_grpc_client import gc
import time
import pandas as pd
import random


In [77]:
controller = Bfrt_GRPC_Client()


Subscribe attempt #1
Subscribe response received 0
Received tf_honeypot on GetForwarding on client 0, device 0
Binding with p4_name tf_honeypot
Binding with p4_name tf_honeypot successful!!


## installation time for different batch size
1. sorted entry (exact match)

In [78]:
IP_INT_MIN = 0
IP_INT_MAX = 2**32 - 1

In [79]:
def test_entry_batch_add(controller, entry_key_list):
    key_list = []
    data_list = []
    for index, key in enumerate(entry_key_list):
        if (key[0], key[1], key[2]) in controller.installed_flow_key:
            continue
        key_list.append(controller.service_table.make_key([gc.KeyTuple("meta.internal_ip", key[0]),
                                                            gc.KeyTuple("meta.internal_port", key[1]),
                                                            gc.KeyTuple("meta.ip_protocol", key[2])]))
        controller.installed_flow_key.add((key[0], key[1], key[2]))
        data_list.append(controller.service_table.make_data([gc.DataTuple('$ENTRY_TTL', 2000)], 
                                                            'Ingress.drop'))
    start_time = time.time()
    controller.service_table.entry_add(controller.target, key_list, data_list)
    end_time = time.time()
    return end_time - start_time

def test_entry_batch_remove(controller, entry_key_list):
    key_list = []
    for index, key in enumerate(entry_key_list):
        if (key[0], key[1], key[2]) in controller.installed_flow_key:
            continue
        key_list.append(controller.service_table.make_key([gc.KeyTuple("meta.internal_ip", key[0]),
                                                    gc.KeyTuple("meta.internal_port", key[1]),
                                                    gc.KeyTuple("meta.ip_protocol", key[2])]))
        controller.installed_flow_key.add((key[0], key[1], key[2]))

    start_time = time.time()
    controller.service_table.entry_del(controller.target, key_list)
    end_time = time.time()
    return end_time - start_time


def test_idle_entry_batch_clean(controller, entry_key_list):

    key_list = []
    count = 0
    retry = 0

    while len(key_list) < len(entry_key_list):
        try:
            time1 = time.time()
            idle_notification = controller.interface.idletime_notification_get(timeout=0.2)
            recv_key = controller.bfrt_info.key_from_idletime_notification(idle_notification)
            key_dict = recv_key.to_dict()
            ip = key_dict["meta.internal_ip"]['value']
            port = key_dict["meta.internal_port"]['value']
            protocol = key_dict["meta.ip_protocol"]['value']
            time2 = time.time()
            controller.installed_flow_key.remove((ip_to_int(ip), port, protocol))

            key_list.append(recv_key)
            count += 1
        except RuntimeError as e:
            print(e, retry)
            retry += 1
            if retry >= 3:
                return -1
        except KeyError as e:
            pass
        
    if len(key_list) > 0:
        time3 = time.time()
        controller.service_table.entry_del(controller.target, key_list)
        time4 = time.time()
    
    return time4 - time3 + time2 - time1

def clean_notification(controller):
    while True:
        try:
            idle_notification = controller.interface.idletime_notification_get(timeout=0.2)
            recv_key = controller.bfrt_info.key_from_idletime_notification(idle_notification)
            key_dict = recv_key.to_dict()
            ip = key_dict["meta.internal_ip"]['value']
            port = key_dict["meta.internal_port"]['value']
            protocol = key_dict["meta.ip_protocol"]['value']
            controller.installed_flow_key.remove((ip_to_int(ip), port, protocol))
        except RuntimeError as e:
            print(e)
            break
        except KeyError as e:
            pass

In [ ]:
controller.clear_service_table()
time_list = []

for experiment_index in range(3):
    for a in range(0, 5):
        for b in range(1, 10):
            controller.clear_service_table()
            
            batch_size = b * pow(10, a)
            if batch_size > 500000:
                break

            test_key_list = []
            for i in range(batch_size):
                test_key_list.append([167772161+i+1, experiment_index*60+a+b, experiment_index*60+a+b])
            
            add_time = test_entry_batch_add(controller, test_key_list)
            print(f"add_time: {add_time}")

            time.sleep(add_time+5)
            del_time = test_entry_batch_remove(controller, test_key_list)
            print(f"del_time: {del_time}")

            controller.clear_service_table()
            clean_notification(controller)

            # one wired thing is that when adding entries just deleted, nothing will be added
            test_key_list = []
            for i in range(batch_size):
                test_key_list.append([167772161+i+1, experiment_index*60+a+b+500, experiment_index*60+a+b])
            
            # test notification
            print("start add again")
            _ = test_entry_batch_add(controller, test_key_list)
            time.sleep(add_time+5)
            print("start clean")
            clean_time = test_idle_entry_batch_clean(controller, test_key_list)
            print(f"clean_time: {clean_time}")
            
            time_list.append({"experiment_index": experiment_index,
                              "batch_size": batch_size, 
                                "add_time": add_time,
                                "del_time": del_time,
                                "clean_time": clean_time,
                                "add_time_per_entry": add_time / batch_size,
                                "del_time_per_entry": del_time / batch_size,
                                "clean_time_per_entry": clean_time / batch_size})
            print(f"batch_size: {batch_size}, add_time: {add_time}, del_time: {del_time}, clean_time: {clean_time}")
    print(f"-----------------------experiment {experiment_index} finished-----------------")

    time_df = pd.DataFrame(time_list)
    time_df.to_csv("./results/grpc_result/entry_add_sorted.csv", index=False)


random entry

In [ ]:
controller.clear_service_table()
time_list = []

for experiment_index in range(3):
    for a in range(0, 5):
        for b in range(1, 10):
            controller.clear_service_table()
            
            batch_size = b * pow(10, a)
            if batch_size > 500000:
                break

            test_key_list = []
            for i in range(batch_size):
                test_key_list.append([random.randint(167772161, 335544321), experiment_index*60+a+b, experiment_index*60+a+b])
            
            add_time = test_entry_batch_add(controller, test_key_list)
            print(f"add_time: {add_time}")

            time.sleep(add_time+5)
            del_time = test_entry_batch_remove(controller, test_key_list)
            print(f"del_time: {del_time}")

            controller.clear_service_table()
            clean_notification(controller)

            # one wired thing is that when adding entries just deleted, nothing will be added
            test_key_list = []
            for i in range(batch_size):
                test_key_list.append([random.randint(503316481, 671088641), experiment_index*60+a+b+500, experiment_index*60+a+b])
            
            # test notification
            print("start add again")
            _ = test_entry_batch_add(controller, test_key_list)
            time.sleep(add_time+20)
            print("start clean")
            clean_time = test_idle_entry_batch_clean(controller, test_key_list)
            print(f"clean_time: {clean_time}")
            
            time_list.append({"experiment_index": experiment_index,
                              "batch_size": batch_size, 
                                "add_time": add_time,
                                "del_time": del_time,
                                "clean_time": clean_time,
                                "add_time_per_entry": add_time / batch_size,
                                "del_time_per_entry": del_time / batch_size,
                                "clean_time_per_entry": clean_time / batch_size})
            print(f"batch_size: {batch_size}, add_time: {add_time}, del_time: {del_time}, clean_time: {clean_time}")
    print(f"-----------------------experiment {experiment_index} finished-----------------")

    time_df = pd.DataFrame(time_list)
    time_df.to_csv("./results/grpc_result/entry_add_random.csv", index=False)


add_time: 0.019280672073364258
del_time: 0.0031833648681640625
Idletime notification not received
start add again
start clean
clean_time: 0.003190279006958008
batch_size: 1, add_time: 0.019280672073364258, del_time: 0.0031833648681640625, clean_time: 0.003190279006958008
add_time: 0.0019800662994384766
del_time: 0.002781391143798828
Idletime notification not received
start add again
start clean
clean_time: 0.003009319305419922
batch_size: 2, add_time: 0.0019800662994384766, del_time: 0.002781391143798828, clean_time: 0.003009319305419922
add_time: 0.0021157264709472656
del_time: 0.002863168716430664
Idletime notification not received
start add again
start clean
clean_time: 0.003039836883544922
batch_size: 3, add_time: 0.0021157264709472656, del_time: 0.002863168716430664, clean_time: 0.003039836883544922
add_time: 0.002040386199951172
del_time: 0.003092050552368164
Idletime notification not received
start add again
start clean
clean_time: 0.003305673599243164
batch_size: 4, add_time: 0

In [75]:
ip_to_int('40.0.0.1')

671088641